In [ ]:
import os
import sys
import json

In [ ]:
with open('eu4.json','rt') as fp:
    data = json.load(fp)
flags = data['f']
pos = data['pos']
prov = data['prov']
state = data['c']

In [ ]:
tmp = sorted(sum([[k for k,v in _.items() if k.count('.')==2] for k,_ in prov.items()],[]))
fy,ly = tmp[0],tmp[-1]
fy,ly
ny = '1444.11.11'

In [ ]:
owners = {}
for p in prov:
    if 'controller' in prov[p]:
        owners[p] = prov[p]['controller']

In [ ]:
e_owner = set()
f_owner = set(owners.values())
for c in f_owner:
    for item in state[c]:
        if item.count('.') == 2:
            e_owner.add(c)
            break

In [ ]:
len(e_owner),len(f_owner)

In [ ]:
events = set()
for sidx,s in state.items():
    for et,e in s.items():
        if et.count('.') == 2:
            if type(e) == list:
                for tmp in e:
                    if type(e) == dict:
                        events.add(list(tmp.keys())[0])
            elif type(e) == dict:
                events.add(list(e.keys())[0])

In [ ]:
# maybe only capital matters in state events

# otherwise go by provinces
# keep track of owners properties
# go by provices 

In [ ]:
event_list = []
for idx,p in prov.items():
    for k,res in p.items():
        if k.count('.') == 2:
            if type(res) == list:
                for tmp in res:
                    if type(tmp) == dict:
                        for typ,evt in tmp.items():
                            if typ in ['owner']:
                                event_list.append((k,idx,typ,evt))
                            elif typ == 'capital':
                                event_list.append((k,idx,'cap2',evt))
                #raise
            elif type(res) == dict:
                for typ,evt in res.items():
                    if typ in ['owner']:
                        event_list.append((k,idx,typ,evt))
                    elif typ == 'capital':
                        event_list.append((k,idx,'cap2',evt))


In [ ]:
for idx,p in state.items():
    for k,res in p.items():
        if k.count('.') == 2:
            #print(res)
            if type(res) == list:
                for tmp in res:
                    if type(tmp) == dict:
                        for typ,evt in tmp.items():
                            if typ in ['capital']:
                                event_list.append((k,idx,typ,evt))
                #raise
            elif type(res) == dict:
                for typ,evt in res.items():
                    if typ in ['capital']:
                        event_list.append((k,idx,typ,evt))

In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial import distance
import numpy as np
from collections import defaultdict

year_events = defaultdict(list)
for event in event_list:
    yr,m,d = event[0].split('.')
    year_events[yr].append(event)
    
def gen_divs(locations):
    keys = sorted(locations.keys())
    values = [locations[k] for k in keys]
    clf = KMeans(6,n_init=1000)
    div_clust = clf.fit_predict(values)


    conf_divs = (clf.cluster_centers_[:,0].argsort().argsort() < 3).astype(int)
    print(conf_divs)
    print(clf.cluster_centers_)
    divs_divs = np.zeros_like(conf_divs)
    for c in range(2):
        cf = np.where(conf_divs == c)
        divs_divs[cf] = (clf.cluster_centers_[cf,1].argsort().argsort()[::-1] + 3*c)[0]
    #divs_divs = sum(divs_divs,[])

    div_lookup = {}
    print(len(keys),len(div_clust))
    for k,di in zip(keys,div_clust):
        div_lookup[k] = (int(conf_divs[di]),int(divs_divs[di]))
    return div_lookup,clf.cluster_centers_,conf_divs,divs_divs
#tmp = gen_divs({k:pos[v]['position'][:2] for k,v in current_loc.items() if k in valid_teams})


In [ ]:
tmp[-1],tmp[-2]

In [ ]:
teams = defaultdict(set)
current_loc = {}
prov_owners = {}
for idx, p in prov.items():
    if 'owner' in p:
        owner = p['owner']
        teams[owner].add(idx)
        prov_owners[idx] = owner
        current_loc[owner] = pos[str(state[owner]['capital'])]['position'][:2]
        
from copy import deepcopy

VALID_TEAMS = set()

for year in sorted(year_events.keys()):
    old_teams = [t for t,i in teams.items() if len(i) >= 10]
    for team in old_teams:
        VALID_TEAMS.add(team)
    for event in sorted(year_events[year]):
        tm, i, t, j = event
        if t =='owner':
            if i in prov_owners:
                teams[prov_owners[i]].remove(i)
            prov_owners[i] = j
            teams[j].add(i)
            if j not in current_loc:
                current_loc[j] = pos[str(i)]['position'][:2]

        elif t == 'capital':
            current_loc[i] = pos[str(j)]['position'][:2]

    new_teams = [t for t,i in teams.items() if len(i) >= 10]

    for team in new_teams:
        VALID_TEAMS.add(team)

In [ ]:
teams = defaultdict(set)
current_loc = {}
current_city = {}
prov_owners = {}
TIDS = {}
for idx, p in prov.items():
    if 'owner' in p:
        owner = p['owner']
        teams[owner].add(idx)
        prov_owners[idx] = owner
        current_loc[owner] = pos[str(state[owner]['capital'])]['position'][:2]
    if 'capital' in p:
        current_city[idx] = p['capital']

In [ ]:
team_hax = {}

valid_teams = {k:v for k,v in teams.items() if k in VALID_TEAMS}
test_label,test_pos,conf_div,div_div = gen_divs({k:v for k,v in current_loc.items() if k in valid_teams})
for team, ps in valid_teams.items():
    if team == '---':
        continue
    d = {}
    d['abbrev'] = team
    if team in flags:
        d['imgURL'] = 'http://eu4.paradoxwikis.com' + flags[team]
    d['name'] = state[team]['name']
    total = 0
    for p in ps:
        total += prov[p]['base_tax'] + prov[p]['base_manpower'] + prov[p]['base_production']
    d['pop'] = round(total/10,2) + 0.1
    d['region'] = current_city[str(state[team]['capital'])]#['name']
    d['stadiumCapacity'] = 20000
    d['cid'] = test_label[team][0]
    d['did'] = test_label[team][1]
    TIDS[team] = len(TIDS)
    d['tid'] = TIDS[team]
    #d['colors'] = color_u2[name]
    team_hax[team] = d


In [ ]:
data = {}
data['teams'] = list(team_hax.values())
import json
data['version'] = 36
data['gameAttributes'] = [    { "key": "startingSeason","value": 1364},{ "key": "season","value": 1364} ]
with open('history_initial.json','wt') as fp:
    json.dump(data,fp, sort_keys=True)

In [ ]:
A = np.array([_['position'][:2] for _ in pos.values()])
import matplotlib.pyplot as plt
plt.scatter(A[:,0],A[:,1],s=1)
plt.scatter(test_pos[:,0],test_pos[:,1],s=400,c=np.array(div_div)%3,cmap='RdYlBu')
conf_div
plt.figure()
plt.hist(A[:,0])

In [ ]:
from copy import deepcopy

roster_events = defaultdict(list)
teamedit = []

for year in sorted(year_events.keys()):
    edraft = []

    old_teams = {t:len(i) for t,i in teams.items() if len(i) >= 0}
    for event in sorted(year_events[year]):
        tm, i, t, j = event
        if t =='owner':
            if i in prov_owners:
                teams[prov_owners[i]].remove(i)
            prov_owners[i] = j
            teams[j].add(i)
            if j not in current_loc:
                current_loc[j] = pos[str(i)]['position'][:2]
        elif t == 'capital' and i in VALID_TEAMS:
            current_loc[i] = pos[str(j)]['position'][:2]
            roster_events[year].append(('cap',i,j))
            d2 = {'type':'teamInfo','season':int(year),'phase':0,'id':len(teamedit)}
            if i not in TIDS:
                TIDS[i] = len(TIDS)
            if j not in current_city:
                current_city[j] = prov[str(j)]['capital'] 
            d2i = {'tid':TIDS[i],'region':current_city[j]}
            d2['info'] = d2i
            teamedit.append(d2)

        elif t == 'cap2':
            current_city[i] = j
            roster_events[year].append(('cap2',i,j))
        else:
            #print(event)
            pass
    new_teams = {t:len(i) for t,i in teams.items() if len(i) >= 0}
    # who disappeared
    for team in old_teams:
        if team not in new_teams and team in VALID_TEAMS:
            roster_events[year].append(('remove',team))
            d2 = {'type':'contraction','season':int(year),'phase':4,'id':len(teamedit), 'info':{'tid':TIDS[team]}}
            teamedit.append(d2)
            print("CONTRACTION")

    # who appeared
    for team in new_teams:
        if team not in old_teams and team in VALID_TEAMS:
            roster_events[year].append(('add',team))
            TIDS[team] = len(TIDS)
    for team in new_teams:
        if team in old_teams and team in VALID_TEAMS:
            if new_teams[team] != old_teams[team]:
                total = 0
                for p in teams[team]:
                    total += prov[p]['base_tax'] + prov[p]['base_manpower'] + prov[p]['base_production']
                roster_events[year].append(('size',team,total))
                d2 = {'type':'teamInfo','season':int(year),'phase':0,'id':len(teamedit)}
                d2i = {'tid':TIDS[team],'pop': round(total/10+ 0.1,2) }
                d2['info'] = d2i
                teamedit.append(d2)
    # add events
    for event in roster_events[year]:
        if event[0] == 'add' and event[1] != '---':
            edraft.append(event[1])
    if len(edraft) > 0:
        d2 = {'type':'expansionDraft','season':int(year),'phase':4,'id':len(teamedit), 'info':{'teams':[]}}
        for team in edraft:
            d = {}
            d['abbrev'] = team
            TIDS[team] = len(TIDS)
            if team in flags:
                d['imgURL'] = 'http://eu4.paradoxwikis.com' + flags[team]
            d['name'] = state[team]['name']
            total = 0
            for p in ps:
                total += prov[p]['base_tax'] + prov[p]['base_manpower'] + prov[p]['base_production']
            d['pop'] = round(total/30+ 0.1,2) 
            d['region'] = current_city[str(state[team]['capital'])]#['name']
            #d['stadiumCapacity'] = 20000
            div = np.linalg.norm(test_pos - current_loc[team],axis=1).argmin()
            d['cid'] = int(conf_div[div])
            d['did'] = int(div_div[div])

            d['tid'] = TIDS[team]
            #d['colors'] = color_u2[name]
            d2['info']['teams'].append(d)
        teamedit.append(d2)

In [ ]:
year_events[year],current_loc['USA']

In [ ]:
[(k,v) for k,v in current_city.items() if v == event[1]]

In [ ]:
year_events['1823']

In [ ]:
teamedit

In [ ]:
data = {}
data['teams'] = list(team_hax.values())
import json
data['version'] = 37
data['gameAttributes'] = [    { "key": "startingSeason","value": 1364},{ "key": "season","value": 1364} ]
data['scheduledEvents'] = teamedit
with open('history_final.json','wt') as fp:
    json.dump(data,fp, sort_keys=True)